Technical Report - Divvy Bike Share Data Analysis
----
Diogo Pessoa
TODO - replace before submission
{$STUDENT_ID}


In [1]:
import os
import sys

from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

images_path = os.getenv('IMAGES_PATH')

## Collect Data

In [2]:
"""
Loading Pandas to validate some checks in sample data, to optimize time.
"""

from divvy_bike_share_data_analysis.data_loader import load_dataset_to_local_fs
from divvy_bike_share_data_analysis.utils_pyspark import load_dataset_to_spark

import glob

# Collecting data to local directory
DATA_COLLECTION_DIR: str = '../data_collection/'

if not glob.glob(os.path.join(DATA_COLLECTION_DIR, '*.csv')):
    print('collecting data to local fs')
    # load_dataset_to_local_fs(DATA_COLLECTION_DIR, [2020, 2021, 2022, 2023])
    # TODO notice to fact that 2020 and 2021, could affect our prediction due to the pandemic (context matters)
    load_dataset_to_local_fs(DATA_COLLECTION_DIR, [2023])

# creating pyspark session and pre-processing data
divvy_df = load_dataset_to_spark(DATA_COLLECTION_DIR)

collecting data to local fs
Requesting file: 202301-divvy-tripdata.zip
Error downloading file status code: 202301-divvy-tripdata.zip, 200
Requesting file: 202302-divvy-tripdata.zip
Error downloading file status code: 202302-divvy-tripdata.zip, 200
Requesting file: 202303-divvy-tripdata.zip
Error downloading file status code: 202303-divvy-tripdata.zip, 200
Requesting file: 202304-divvy-tripdata.zip
Error downloading file status code: 202304-divvy-tripdata.zip, 200
Requesting file: 202305-divvy-tripdata.zip
Error downloading file status code: 202305-divvy-tripdata.zip, 200
Requesting file: 202306-divvy-tripdata.zip
Error downloading file status code: 202306-divvy-tripdata.zip, 200
Requesting file: 202307-divvy-tripdata.zip
Error downloading file status code: 202307-divvy-tripdata.zip, 200
Requesting file: 202308-divvy-tripdata.zip
Error downloading file status code: 202308-divvy-tripdata.zip, 200
Requesting file: 202309-divvy-tripdata.zip
Error downloading file status code: 202309-divvy-

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/06 12:41:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


FileNotFoundError: Schema file ../data_collection/divvy-tripdata-schema.yaml does not exist.

## Sampling data for local validation

In [3]:
sampled_df = divvy_df.sample(withReplacement=False, fraction=0.01)


## Data Preprocessing

* Verifying bike stations have correct name and id mapping for ech trip start and end stations
* Remove Duplicate, trip records have Unique Trip Ids. Drop rows with null and duplicates values for `ride_id` 

In [4]:
# removing duplicates and null values from ride_id and station Ids
sampled_df = sampled_df.dropDuplicates(subset=['ride_id']).dropna(subset=['ride_id'])

In [5]:
# Get unique bike stations, removing rows with null values for station Ids
from divvy_bike_share_data_analysis.bike_stations import get_unique_bike_stations_ids
sampled_df = sampled_df.dropna(subset=['start_station_id', 'end_station_id'])
sampled_df.show(10)
bike_stations = get_unique_bike_stations_ids(sampled_df.select(['start_station_id',
                                                        'start_station_name',
                                                        'end_station_id',
                                                        'end_station_name']))

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+---------+---------+---------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|start_lat|start_lng|  end_lat|  end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+---------+---------+---------+-------------+
|00005F98B137E17C|  docked_bike|2021-08-22 21:59:28|2021-08-22 23:12:35|Franklin St & Ill...|             RN-|Franklin St & Ill...|           RN-| 41.89102|-87.63548| 41.89102|-87.63548|       casual|
|000062A9C95F1CA1| classic_bike|2022-10-28 13:31:38|2022-10-28 13:36:36| Morgan St & Polk St|    TA1307000130|Loomis St & Lexin...|         13332| 41.87174|-87.65103| 41.87219| -87.6615|       mem

### Applying unified Station Names by ID back to sampled dataset

In [6]:
sampled_df.show(10)

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+---------+---------+---------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|start_lat|start_lng|  end_lat|  end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+---------+---------+---------+-------------+
|00005F98B137E17C|  docked_bike|2021-08-22 21:59:28|2021-08-22 23:12:35|Franklin St & Ill...|             RN-|Franklin St & Ill...|           RN-| 41.89102|-87.63548| 41.89102|-87.63548|       casual|
|000062A9C95F1CA1| classic_bike|2022-10-28 13:31:38|2022-10-28 13:36:36| Morgan St & Polk St|    TA1307000130|Loomis St & Lexin...|         13332| 41.87174|-87.65103| 41.87219| -87.6615|       mem

In [7]:
from divvy_bike_share_data_analysis.bike_stations import categorize_time_of_day, categorize_day_of_week
from pyspark.sql.functions import udf, hour, dayofweek
from pyspark.sql.types import StringType

# TODO refactoring to optmized queries and avoid duplicated columns after joins.
# [Note to self] check bike_stations.get_unique_ids function for similar approach.

# Prepare start and end stations DataFrames from bike_stations
start_stations = bike_stations.selectExpr("station_id as start_station_id", "station_name as new_start_station_name")
end_stations = bike_stations.selectExpr("station_id as end_station_id", "station_name as new_end_station_name")

# Drop existing name columns in sampled_df before joining
sampled_df = sampled_df.drop("start_station_name", "end_station_name")

# Join with start_stations to add new_start_station_name
sampled_df = sampled_df.join(start_stations, on="start_station_id", how="left").withColumnRenamed("new_start_station_name", "start_station_name")

# Join with end_stations to add new_end_station_name
sampled_df = sampled_df.join(end_stations, on="end_station_id", how="left").withColumnRenamed("new_end_station_name", "end_station_name")

## Feature Engineering

### Adding new features 

As discussed in the [Technical Report](Reports/TechReport/out/Technical_Report_Diogo_Pessoa.pdf).
 
I'll add a categorical field for time of day and day of week. Later I'll use these fields to analyze the most popular stations by time of day and day of week.
For both Categorical fields the idea is to simplify the data and make it easier to analyze. While also labeling trips and opening an avenue for further exploration in regression models
* Time of day: Morning, Afternoon, Evening, Night  
* Day of week: Workday, non-Working

In [8]:
# Adding a categorical field for time of day and day of week.
week_day_udf = udf(categorize_day_of_week, StringType())
time_of_day_udf = udf(categorize_time_of_day, StringType())
sampled_df_with_added_features = sampled_df.withColumn('day_period', time_of_day_udf(hour('started_at'))).withColumn(
    'week_day', week_day_udf(dayofweek('started_at')))

### Data exploration

Section dedicated to exploring the data and validating the added features. Using Visualizations to understand the aggregated data and the relationships between the chosen features.

#### Most Popular Start Stations

In [ ]:
from pyspark.sql.functions import desc

# Count Start stations
# Group by 'start_station_id' and count
station_counts = sampled_df_with_added_features.groupBy('start_station_name').count()
print(station_counts) #TODO remove this print - debug only

"""
Converting to Pandas DataFrame to load into seaborn for plotting. 
Note I'm only converting to Pandas after the PySpark aggregations, aware of the performance implications in larger datasets. 
"""
top_10_stations = station_counts.orderBy(desc('count')).limit(10).toPandas()

DataFrame[start_station_name: string, count: bigint]


In [ ]:
top_10_stations.head(10) #start stations
# plot top ten stations, end. By day period. 
# Further explore the correlation start end, by day period, filtering by week day.
# Plot a Correlation matrix of the features before applying PCA
# TODO -  Stick to the fact that the goal in this project is to optimize the service to restock the stations, avoiding empty stations or users waiting to dock the bikes

In [19]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 8))
ax = sns.countplot(
    data=top_10_stations,
    x='start_station_id',
    palette='viridis',
)

plt.title('Most Popular Start Stations')
plt.xlabel('Start Station Names')
plt.ylabel('Count By Trips')
plt.xticks(rotation=45)  # Rotate the x-tick labels for better readability
plt.show()
# Save the plot to TechReport images folder
plt.savefig(os.path.join(images_path, 'count_most_popular_start_stations.png'))

NameError: name 'top_10_stations' is not defined

<Figure size 1000x800 with 0 Axes>

## [Unsupervised Learning] Predictive Analysis (Kmeans Clustering)

Using the added features, I'll apply K-means clustering to find the most popular source stations by time of day.

The steps involve: 
#### Data Preparation:
##### Convert the categorical features to numeric index (StringIndexer)
* Potentially use `OneHotEncoder`, since, we added the categorical fields for time of day and day of week, we may skip this step.
* Vectorize the features (VectorAssembler)
* Apply PCA to inspect the relationships between the chosen features, for correlation and variance.
 
##### Clustering:

Run an initial KMeans clustering with a default of k=5 clusters, then apply the silhouette index to look for the optimal number of clusters. 
Compare and plot results.

Proceed with k-means for some specific questions:

* Apply K-means clustering to find the most popular source/destinations stations - Plot  
* Apply K-means for most popular stations by time of day/day of week - Plot 
* Most popular routes by day of week - Plot
* Most popular routes by day of week/time of day - Plot

In a real-world scenario, the period of the day (Morning, Afternoon, Evening, and Night) is not granular enough. To get better accuracy in the prediction, to confidently allocate, employees to relocate bikes. We should consider shorter time windows, increasing  granularity, such as 15-minute intervals.

#### [Supervised Learning] Predictive Analysis (Regression)

Explore a predictive analysis using regression models to improve user experience. Refining the re-stock strategy by considering the bike type per route, to predict the number of bikes needed at each station at different times of the day.

* Apply regression models to predict the number of bikes needed at each station at different times of the day.
* Bike type per routes, to predict the number of bikes needed at each station at different times of the day.

In [ ]:

# features = ['start_station_id', 'end_station_id', 'day_period', 'week_day']
# # df = divvy_df_pandas[['start_station_id_index', 'end_station_id_index', 'day_period_index', 'week_day_index']]
# df = pd.DataFrame(divvy_df_pandas[features], columns=features)

In [ ]:
# # Plotting correlation matrix of scaled set
# import seaborn as sns
# import matplotlib.pyplot as plt
# 
# corr = df.corr()
# # Plot the heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(corr, annot=True, fmt=".2f", cmap='inferno', linewidth=.5, cbar=True, square=True)
# plt.title('Feature Correlation Matrix')
# plt.show()

In [ ]:
# # Convert Station id to numeric Index Pandas 
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import StandardScaler
# import pandas as pd
# 
# le = LabelEncoder()
# le.fit(divvy_df_pandas['start_station_id'])
# divvy_df_pandas['start_station_id_index'] = le.transform(divvy_df_pandas['start_station_id'])
# 
# # Convert end Station id to numeric Index Pandas 
# le.fit(divvy_df_pandas['end_station_id'])
# divvy_df_pandas['end_station_id_index'] = le.transform(divvy_df_pandas['end_station_id'])
# 
# # Convert day period to numeric index Pandas
# le.fit(divvy_df_pandas['day_period'])
# divvy_df_pandas['day_period_index'] = le.transform(divvy_df_pandas['day_period'])
# 
# # Convert week day to numeric index Pandas
# le.fit(divvy_df_pandas['week_day'])
# divvy_df_pandas['week_day_index'] = le.transform(divvy_df_pandas['week_day'])

In [ ]:
# # confusion matrix preparing to review the most popular stations by time of day. 
# # This is ahead of a PCA analysis the relationships between the chosen features.
# 
# from sklearn.metrics import confusion_matrix
# from matplotlib import pyplot as plt
# import seaborn as sns
# 
# # Scale the features
# scaler = StandardScaler()
# df_scaled = scaler.fit_transform(df)
# 
# cm = confusion_matrix(features, df_scaled)
# 
# # Plotting the confusion matrix
# plt.figure(figsize=(10, 7))  # Adjust the size as needed
# 
# # Using seaborn to create an annotated heatmap
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
# plt.title('Confusion Matrix')
# plt.ylabel('Actual Label')
# plt.xlabel('Predicted Label')
# 
# # Adjust tick labels if necessary
# # plt.xticks(np.arange(len(your_tick_labels)), your_tick_labels, rotation=45)
# # plt.yticks(np.arange(len(your_tick_labels)), your_tick_labels, rotation=45)
# plt.show()

In [ ]:
# from sklearn.decomposition import PCA
# 
# import pandas as pd
# 
# # Standardizing the features
# 
# 
# """
# The goal here is to inspect the relationships between the chosen features, for correlation and variance.
# PCA: https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_iris.html#sphx-glr-auto-examples-decomposition-plot-pca-iris-py 
# """
# # Applying PCA
# pca = PCA(n_components=2)
# principalComponents = pca.fit_transform(df_scaled)
# principalDf = pd.DataFrame(data=principalComponents, columns=['Principal Component 1', 'Principal Component 2'])
# 
# # Print results
# print(pca.explained_variance_ratio_)
# # Plotting
# 
# import matplotlib.pyplot as plt
# 
# plt.figure(figsize=(8, 6))
# plt.scatter(principalDf['Principal Component 1'], principalDf['Principal Component 2'])
# plt.xlabel('Principal Component 1')
# plt.ylabel('Principal Component 2')
# plt.title('2 component PCA')
# plt.show()


In [ ]:
# # Repeat the Confusion Matrix with the PCA results
# cm = confusion_matrix(principalDf['Principal Component 1'], principalDf['Principal Component 2'])
# 
# # Plotting the confusion matrix
# 
# plt.figure(figsize=(10, 7))  # Adjust the size as needed
# 
# # Using seaborn to create an annotated heatmap
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
# plt.title('Confusion Matrix')
# plt.ylabel('Actual Label')
# plt.xlabel('Predicted Label')

In [ ]:
# # Run Starting Station KMeans Clustering based on day periods
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler
# import numpy as np
# import pandas as pd
# 
# df = pd.DataFrame(divvy_df_pandas, columns=['start_station_id_index', 'day_period_index'])
# 
# # I'll test scaling the features: and see the outcome
# scaler = StandardScaler()
# df_scaled = scaler.fit_transform(df)
# 
# kmeans = KMeans(n_clusters=11)
# kmeans.fit(df)
# df['start_station_cluster'] = kmeans.predict(df)

In [ ]:
# # Plot kmeans results
# import seaborn as sns
# import matplotlib.pyplot as plt
# 
# plt.figure(figsize=(10, 6))
# sns.scatterplot(data=divvy_df_pandas, x='start_station_id', y='day_period_index', hue='start_station_cluster',
#                 palette='viridis')
# plt.title('KMeans Clustering Results')
# plt.xlabel('Start Station ID')
# plt.xlabel('day_period_index')
# plt.show()

In [ ]:
# import pandas as pd
# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score
# from sklearn.preprocessing import StandardScaler
# 
# # After a default of k=5 clusters, I'll apply the silhouette index to look for the optimal number of clusters
# 
# df = pd.DataFrame(divvy_df_pandas, columns=['start_station_id_index', 'day_period_index'])
# 
# # I'll test scaling the features: and see the outcome
# scaler = StandardScaler()
# df_scaled = scaler.fit_transform(df)
# 
# k_values = range(2, 15)
# silhouette_scores = []
# 
# for k in k_values:
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans.fit(df_scaled)
# 
#     # Predict the cluster labels
#     labels = kmeans.labels_
# 
#     # Calculate the silhouette score and append it to the list
#     silhouette_avg = silhouette_score(df_scaled, labels)
#     silhouette_scores.append(silhouette_avg)
#     print(f"Silhouette Score for k={k}: {silhouette_avg}")
# 
# # Find the k with the highest silhouette score
# optimal_k = k_values[silhouette_scores.index(max(silhouette_scores))]
# print(f"The optimal number of clusters k is: {optimal_k}")


In [ ]:
# # Plotting
# plt.figure(figsize=(10, 6))
# plt.plot(k_values, silhouette_scores, 'bx-')
# plt.xlabel('k')
# plt.ylabel('Silhouette score')
# plt.title('Silhouette score vs. Number of clusters (k)')
# plt.xticks(np.arange(min(k_values), max(k_values) + 1, 1.0))
# plt.grid(True)
# plt.show()
# 
# # Output the optimal k based on silhouette score
# optimal_k = k_values[silhouette_scores.index(max(silhouette_scores))]
# print(f"The optimal number of clusters k is: {optimal_k}")

In [ ]:
# from pyspark.ml.feature import StringIndexer
# 
# indexer_start_station_id = StringIndexer(inputCol="start_station_id", outputCol="start_station_id_index")
# indexer_end_station_id = StringIndexer(inputCol="end_station_id", outputCol="end_station_id_index")
# sanitized_bike_stations_id_indexed = indexer_start_station_id.fit(sanitized_bike_stations).transform(
#     sanitized_bike_stations)
# sanitized_bike_stations_id_indexed = indexer_end_station_id.fit(sanitized_bike_stations).transform(
#     sanitized_bike_stations_id_indexed)
# sanitized_bike_stations_id_indexed.show()
# sanitized_bike_stations = sampled_df_with_day_period_indexed.join(
#     bike_stations.withColumnRenamed("station_name", "start_station_name"),
#     sampled_df_with_day_period_indexed.start_station_id == bike_stations.station_id,
#     "left").drop("station_id")
# 
# # Join to map end_station_id to end_station_name
# sanitized_bike_stations = sanitized_bike_stations.join(
#     bike_stations.withColumnRenamed("station_name", "end_station_name"),
#     sanitized_bike_stations.end_station_id == bike_stations.station_id,
#     "left").drop("station_id")

In [ ]:
# TODO Erroring out, need to fix this
# from pyspark.ml.feature import VectorAssembler
# 
# # most popular stations by period of day
# feature_cols = ['start_station_id_index', 'day_period_index']
# assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
# 
# # Transform the data to have the features column
# adf_kmeans = assembler.transform(sanitized_bike_stations_id_indexed)
# ### Custom spark Session to support synapsesML:
# import pyspark
# 
# spark = pyspark.sql.SparkSession.builder.appName("MyApp") \
#     .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:1.0.2") \
#     .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven") \
#     .getOrCreate()
# 
# from pyspark.ml.clustering import KMeans
# 
# # Number of clusters
# k = 5  # Adjust based on your data and needs
# 
# kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
# model = kmeans.fit(adf_kmeans)

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# 
# # Make predictions
# # predictions = model.transform(adf_kmeans)
# # 
# # # Group by cluster label and start_station_id, and count occurrences
# # popular_start_stations = predictions.groupBy("prediction", "start_station_id_index")
# # 
# # # Find the most popular station in each cluster
# # most_popular_start_stations = popular_start_stations.groupBy("prediction").max("count")
# # 
# # popular_destinations = predictions.groupBy("prediction", "end_station_id_index").count()
# # most_popular_destination = popular_start_stations.groupBy("prediction").max("count")
# 
# # TODO apply K-means clustering and to find the most popular source stations - Plot
# # TODO - Can I apply K-means for most popular stations by time of day? - Plot
# # TODO - most popular routes - Plot 
# 
# 
# predictions.columns
# 
# plt.figure(figsize=(10, 6))
# sns.scatterplot(data=predictions, x='start_station_id_index', y='start_lat', hue='prediction', palette='viridis')
# # plt.title('KMeans Clustering Results')
# # plt.show()

In [ ]:
# """
# Will use this confusion Matrix when I get to the Categorical analysis of the data. 
# """
# # confusion matrix preparing to review the most popular stations by time of day. 
# # This is ahead of a PCA analysis the relationships between the chosen features.
# 
# from sklearn.metrics import confusion_matrix
# from matplotlib import pyplot as plt
# import seaborn as sns
# 
# # Scale the features
# scaler = StandardScaler()
# df_scaled = scaler.fit_transform(df)
# 
# cm = confusion_matrix(features, df_scaled)
# 
# # Plotting the confusion matrix
# plt.figure(figsize=(10, 7))  # Adjust the size as needed
# 
# # Using seaborn to create an annotated heatmap
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
# plt.title('Confusion Matrix')
# plt.ylabel('Actual Label')
# plt.xlabel('Predicted Label')
# 
# # Adjust tick labels if necessary
# # plt.xticks(np.arange(len(your_tick_labels)), your_tick_labels, rotation=45)
# # plt.yticks(np.arange(len(your_tick_labels)), your_tick_labels, rotation=45)
# plt.show()
